In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import time
import os
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
from sqlite3 import connect

In [20]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": "C:\\Users\\gualas\\Desktop\\prueba",
})

In [21]:
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)
# Acceder a la página nro1
driver.get("https://ourworldindata.org/world-population-growth")

time.sleep(5)
# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()

# Acceder a la página nro2
driver.get("https://ourworldindata.org/migration")

time.sleep(5)
# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real                                      
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div/div/div/section/figure[1]/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()

time.sleep(2)
driver.quit()

C:\Users\gualas\AppData\Local\Temp\ipykernel_3080\2521872399.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


In [24]:
paises_migraciones=pd.read_csv(r'C:\Users\gualas\Desktop\prueba\migration.csv')
paises_poblacion=pd.read_csv(r'C:\Users\gualas\Desktop\prueba\population-and-demography.csv')

conn = connect(':memory:') 
paises_migraciones.to_sql('pam',conn,index=False)
paises_poblacion.to_sql('pap',conn,index=False)

paises_migraciones_america = pd.read_sql('SELECT Year,Country,Emigrants,`International migrants`, `Net migration`, `Net migration rate` FROM pam WHERE Country IN ("Argentina","Belice","Bolivia", "Brasil","Canada","Colombia", "Costa Rica", "Cuba", "Chile", "Ecuador", "El Salvador", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru","Puerto Rico", "Republica Dominicana","United States","Uruguay","Venezuela","Antigua y Barbuda","Bahamas","Barbados","Dominica","Granada","Guyana","Jamaica","República Dominicana","San Cristobal y Nieves","San Vicente y las Granadinas","Santa Lucia")', conn)

paises_poblacion_america=pd.read_sql('SELECT `Country name`, Year, Population FROM pap WHERE `Country name` IN ("Argentina","Bolivia","Belice", "Brasil","Canada","Colombia", "Costa Rica", "Cuba", "Chile", "Ecuador", "El Salvador", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru","Puerto Rico", "Republica Dominicana","United States","Uruguay","Venezuela","Antigua y Barbuda","Bahamas","Barbados","Dominica","Granada","Guyana","Jamaica","República Dominicana","San Cristobal y Nieves","San Vicente y las Granadinas","Santa Lucia")',conn)

paises_migraciones_america.to_sql('a',conn,index=False)
paises_poblacion_america.to_sql('c',conn,index=False)

tabla=pd.read_sql('SELECT a.Year,a.Country,a.Emigrants,a.`International migrants`,a.`Net migration`,a.`Net migration rate`,c.Population FROM a a JOIN c c ON(a.Year=c.Year AND a.`Country`=c.`Country name` )',conn)

tabla.dropna(inplace=True)
tabla.reset_index(drop=True, inplace=True)

tabla['relative migration']=tabla['Emigrants']/tabla['Population']
tabla['Inmigrants']=tabla['Emigrants']+tabla['Net migration']

In [25]:
tabla

,Year,Country,Emigrants,International migrants,Net migration,Net migration rate,Population,relative migration,Inmigrants
0,1990,Argentina,430155.0,1649919.0,160000.0,1.019,32637658,0.013180,590155.0
1,1995,Argentina,481968.0,1595069.0,-105000.0,-0.623,34946116,0.013792,376968.0
2,2000,Argentina,555542.0,1540219.0,-128000.0,-0.714,37070772,0.014986,427542.0
3,2005,Argentina,805291.0,1674163.0,-125000.0,-0.660,39070504,0.020611,680291.0
4,2010,Argentina,938639.0,1805957.0,-120000.0,-0.602,41100124,0.022838,818639.0
...,...,...,...,...,...,...,...,...,...
170,2000,Venezuela,317753.0,1013738.0,-4540.0,-0.039,24427732,0.013008,313213.0
171,2005,Venezuela,437087.0,1076474.0,-22740.0,-0.180,26668790,0.016389,414347.0
172,2010,Venezuela,560528.0,1347347.0,-200220.0,-1.460,28715026,0.019520,360308.0
173,2015,Venezuela,709140.0,1404448.0,-431040.0,-2.946,30529722,0.023228,278100.0


In [26]:
# Acceder a la página nro3
#no se encuentra el archivo de descarga data inflacion % 
inflacion_argentina=pd.read_csv(r'inflacion\inflacion_argentina.csv')
inflacion_bahamas=pd.read_csv(r'inflacion\inflacion_bahamas.csv')
inflacion_barbados=pd.read_csv(r'inflacion\inflacion_barbados.csv')
inflacion_bolivia=pd.read_csv(r'inflacion\inflacion_bolivia.csv')
inflacion_brazil=pd.read_csv(r'inflacion\inflacion_brazil.csv')
inflacion_canada=pd.read_csv(r'inflacion\inflacion_canada.csv')
inflacion_chile=pd.read_csv(r'inflacion\inflacion_chile.csv')
inflacion_colombia=pd.read_csv(r'inflacion\inflacion_colombia.csv')
inflacion_costarica=pd.read_csv(r'inflacion\inflacion_costarica.csv')
inflacion_ecuador=pd.read_csv(r'inflacion\inflacion_ecuador.csv')
inflacion_elsalvador=pd.read_csv(r'inflacion\inflacion_elsalvador.csv')
inflacion_guatemala=pd.read_csv(r'inflacion\inflacion_guatemala.csv')
inflacion_guyana=pd.read_csv(r'inflacion\inflacion_guyana.csv')
inflacion_haiti=pd.read_csv(r'inflacion\inflacion_haiti.csv')
inflacion_honduras=pd.read_csv(r'inflacion\inflacion_honduras.csv')
inflacion_jamaica=pd.read_csv(r'inflacion\inflacion_jamaica.csv')
inflacion_mexico=pd.read_csv(r'inflacion\inflacion_mexico.csv')
inflacion_nicaragua=pd.read_csv(r'inflacion\inflacion_nicaragua.csv')
inflacion_panama=pd.read_csv(r'inflacion\inflacion_panama.csv')
inflacion_paraguay=pd.read_csv(r'inflacion\inflacion_paraguay.csv')
inflacion_peru=pd.read_csv(r'inflacion\inflacion_peru.csv')
inflacion_uruguay=pd.read_csv(r'inflacion\inflacion_uruguay.csv')
inflacion_usa=pd.read_csv(r'inflacion\inflacion_usa.csv')
inflacion_venezuela=pd.read_csv(r'inflacion\inflacion_venezuela.csv')


In [27]:
inflacion_argentina.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_argentina['country']='Argentina'
inflacion_bahamas.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_bahamas['country']='Bahamas'
inflacion_barbados.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_barbados['country']='Barbados'
inflacion_brazil.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_brazil['country']='Brazil'
inflacion_bolivia.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_bolivia['country']='Bolivia'
inflacion_canada.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_canada['country']='Canada'
inflacion_chile.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_chile['country']='Chile'
inflacion_colombia.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_colombia['country']='Colombia'
inflacion_costarica.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_costarica['country']='Costa Rica'
inflacion_ecuador.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_ecuador['country']='Ecuador'
inflacion_elsalvador.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_elsalvador['country']='El Salvador'
inflacion_guatemala.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_guatemala['country']='Guatemala'
inflacion_guyana.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_guyana['country']='Guyana'
inflacion_haiti.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_haiti['country']='Haiti'
inflacion_honduras.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_honduras['country']='Honduras'
inflacion_jamaica.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_jamaica['country']='Jamaica'
inflacion_mexico.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_mexico['country']='Mexico'
inflacion_nicaragua.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_nicaragua['country']='Nicaragua'
inflacion_panama.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_panama['country']='Panama'
inflacion_paraguay.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_paraguay['country']='Paraguay'
inflacion_peru.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_peru['country']='Peru'
inflacion_uruguay.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_uruguay['country']='Uruguay'
inflacion_usa.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_usa['country']='United States'
inflacion_venezuela.drop(['web-scraper-order','web-scraper-start-url'],axis=1,inplace=True)
inflacion_venezuela['country']='Venezuela'

In [28]:
prueba = pd.concat([inflacion_argentina,inflacion_bahamas,inflacion_barbados,inflacion_bolivia,inflacion_brazil,inflacion_canada,inflacion_chile,inflacion_colombia,inflacion_costarica,inflacion_ecuador,inflacion_elsalvador,inflacion_guatemala,inflacion_guyana,inflacion_haiti,inflacion_honduras,inflacion_jamaica,inflacion_mexico,inflacion_nicaragua,inflacion_panama,inflacion_paraguay,inflacion_peru,inflacion_uruguay,inflacion_usa,inflacion_venezuela], axis=0,ignore_index=True)

In [29]:
prueba['inflacion']=prueba['inflacion'].apply(lambda x: x.replace("%",""))
prueba['inflacion']=prueba['inflacion'].apply(lambda x: x.replace(".",""))
prueba['inflacion']=prueba['inflacion'].apply(lambda x: x.replace(",","."))

In [30]:
conn = connect(':memory:')
tabla.to_sql('tab',conn,index=False)
prueba.to_sql('pru',conn,index=False)

1347

In [31]:
tabla_2=pd.read_sql('SELECT tab.*,pru.inflacion as Inflation FROM tab tab JOIN pru pru ON(tab.Year=pru.Año AND tab.Country=pru.country)',conn)

In [40]:
# Acceder a la página nro4
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


# Abrir la página de la URL dada
driver.get("https://datos.bancomundial.org/indicador/NY.GDP.MKTP.KD.ZG")

# Esperar a que el botón de descarga esté disponible y hacer clic en él
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainChart"]/aside/div/div[2]/div/p/a[1]')))
boton_descarga.click()

time.sleep(2)
driver.quit()

C:\Users\gualas\AppData\Local\Temp\ipykernel_3080\740268378.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


In [ ]:
pib=pd.read_csv(r'pib\pib.csv')

In [ ]:
pib=pib[pib['Country Name'].isin(['Argentina', 'Bahamas', 'Barbados', 'Bolivia', 'Brazil', 'Canada',
       'Chile', 'Colombia', 'Costa Rica', 'Ecuador', 'El Salvador',
       'Guatemala', 'Guyana', 'Haiti', 'Honduras', 'Jamaica', 'Mexico',
       'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'United States',
       'Uruguay', 'Venezuela'])]
pib=pib.loc[:,['Country Name','1990','1995','2000',
       '2005','2010',  '2015','2020']]
pib.reset_index(inplace=True,drop=True)

In [ ]:
PBI=pd.DataFrame({
    'Country':['Argentina','Argentina','Argentina','Argentina','Argentina','Argentina','Argentina','Bolivia','Bolivia','Bolivia','Bolivia','Bolivia','Bolivia','Bolivia','Brazil','Brazil','Brazil','Brazil','Brazil','Brazil','Brazil','Barbados','Barbados','Barbados','Barbados','Barbados','Barbados','Barbados','Canada','Canada','Canada','Canada','Canada','Canada','Canada','Chile','Chile','Chile','Chile','Chile','Chile','Chile','Colombia','Colombia','Colombia','Colombia','Colombia','Colombia','Colombia','Costa Rica','Costa Rica','Costa Rica','Costa Rica','Costa Rica','Costa Rica','Costa Rica','Ecuador','Ecuador','Ecuador','Ecuador','Ecuador','Ecuador','Ecuador','Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','Guyana','Guyana','Guyana','Guyana','Guyana','Guyana','Guyana','Honduras','Honduras','Honduras','Honduras','Honduras','Honduras','Honduras','Haiti','Haiti','Haiti','Haiti','Haiti','Haiti','Haiti','Jamaica','Jamaica','Jamaica','Jamaica','Jamaica','Jamaica','Jamaica','Mexico','Mexico','Mexico','Mexico','Mexico','Mexico','Mexico','Nicaragua','Nicaragua','Nicaragua','Nicaragua','Nicaragua','Nicaragua','Nicaragua','Panama','Panama','Panama','Panama','Panama','Panama','Panama','Peru','Peru','Peru','Peru','Peru','Peru','Peru','Paraguay','Paraguay','Paraguay','Paraguay','Paraguay','Paraguay','Paraguay','El Salvador','El Salvador','El Salvador','El Salvador','El Salvador','El Salvador','El Salvador','Uruguay','Uruguay','Uruguay','Uruguay','Uruguay','Uruguay','Uruguay','United States','United States','United States','United States','United States','United States','United States','Venezuela','Venezuela','Venezuela','Venezuela','Venezuela','Venezuela','Venezuela'],
    'Year':['1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020','1990', '1995', '2000', '2005', '2010', '2015', '2020'],
    'PBI':[ -2.46721377810472, -2.84520961057079,
        -0.78899893905691, 8.85165992013437, 10.1253981561002,
        2.73115982828944, -9.94323513446069, 4.6357874278518, 4.67827659709971, 2.50781080899483,
        4.42143312687979, 4.1267225910567, 4.85718730035796,
        -8.73788443399144, -4.35000000000022, 4.22379363350413, 4.38794944745618,
        3.20213206130431, 7.52822581815364, -3.54576339269425,
        -3.87867633397198, -3.30759330759331, 2.01399800028568,
        4.44718962322422, 3.95545760352627, -2.36516470308347,
        2.47261987248125, -13.3110215258234, 0.2, 2.7, 5.17763566824114, 3.20420042827129,
        3.08949461982786, 0.659176863558869, -5.23302430280606, 3.33356547444561, 8.93329586932387, 4.9716214882971,
        5.83704571917758, 5.85165102224047, 2.15194249981184,
        -5.9782240617438, 4.28199833053058, 5.2024375925092, 2.92486148314592,
        4.82876110795085, 4.49465897070922, 2.95590137527523,
        -7.04815120786543, 3.55306336572285, 4.15395304305277,
        3.86871808818643, 3.97666636792908, 5.36034347985841,
        3.65208096802883, -4.05090817067276, 3.67991405278418, 2.25254877423716, 1.0918015643576,
        5.29130826699402, 3.52529866894027, 0.0988726083462694,
        -7.78760709093422, 3.10256317557587, 4.94854750210382, 3.6088687174876,
        3.27602692588886, 2.88417542800377, 4.09217081621244,
        -1.76408292563957, -3.06748466406728, 5.03144655812085, -1.36380390261873,
        -1.95095759501714, 4.13844160137342, 0.687521210092171,
        43.4795559399402, 2.7917489495977, 6.18930455759235, 7.29128815869618,
        6.05059922215973, 3.73114034433009, 3.84007997093951,
        -8.96476018404584, 1.03812280198248, 9.89770748815705, 0.870059259424892,
        3.07217337664731, -5.65151153448299, 2.5625088349003,
        -3.34337261175989, 4.2005351147029, 2.34995704147829, 0.87870354796236,
        0.893797935945003, -1.45714720947502, 0.92148712538031, -10.0, 5.17576838681073, -6.29123082110117, 4.94245371467422,
        2.30780706591736, 5.11811814321163, 3.29315152833387,
        -7.98791234014749, -0.0524997212687737, 5.91190795714505,
        4.10159023598223, 4.28239836333562, 4.40964549908733,
        4.79205676554646, -1.7899754441716, 8.09899300573198, 1.7516787571155, 2.71537415482821,
        7.19127940233535, 5.82799349428571, 5.7330542960121,
        -17.9448935972493, -4.98256353646605, 7.41139504735119, 2.69437139806912,
        6.28506032509601, 8.33245910749577, 3.25224477218451,
        -10.9526989325015, 4.12328287941739, 6.82281030358578,
        -2.31414056819045, 2.13349066459601, 11.0952312685521,
        2.95715173487106, -0.819785390971916, 4.83290057793786, 4.73408140229732,
        1.12790697674419, 2.70546347259486, 2.10788757932909,
        2.40000000000001, -8.17721743762763, 0.297348378681562, -1.44759896894408,
        -1.92993063772035, 7.46013213775913, 7.80340966564692,
        0.370741265001499, -6.12147627999001, 1.88596495668033, 2.68421727207786,
        4.07715947941618, 3.48322032025258, 2.70885669419681,
        2.70636958176399, -2.76780251140578,6.5,4,3.7,10.3,-1.5,-15.7,-19.7]})

In [ ]:
PBI.to_sql('pbi',conn,index=False)
tabla_2.to_sql('t2',conn,index=False)
tabla_3=pd.read_sql('SELECT t2.*,pbi.PBI FROM t2 t2 JOIN pbi pbi ON(t2.Year=pbi.Year AND t2.Country=pbi.Country)',conn)

In [14]:
# Acceder a la página nro5 poblacion femenina 
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)

# Abrir la página de la URL dada
driver.get("https://ourworldindata.org/world-population-growth")

# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/form/div[2]/div[3]/label/input')))
boton_descarga.click()

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()

time.sleep(2)
driver.quit()

C:\Users\gualas\AppData\Local\Temp\ipykernel_3080\3520641847.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


In [ ]:
poblacion_mujeres=pd.read_csv(r'rangos/population-and-demography_female.csv')
poblacion_mujeres.to_sql('muj',conn,index=False)
tabla_3.to_sql('t3',conn,index=False)
tabla_4=pd.read_sql('SELECT t3.*,muj.`Female population`,muj.`Female population aged 20 to 29 years`,muj.`Female population aged 30 to 39 years`,muj.`Female population aged 40 to 49 years`,muj.`Female population aged 50 to 59 years`,muj.`Female population aged 60 to 69 years` FROM t3 t3 JOIN muj muj ON(t3.Year=muj.Year AND t3.Country=muj.`Country name`)',conn)

In [15]:
# Acceder a la página nro6 poblacion masculina
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


# Abrir la página de la URL dada
driver.get("https://ourworldindata.org/world-population-growth")

# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/form/div[2]/div[4]/label/input')))
boton_descarga.click()

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()

time.sleep(2)
driver.quit()

C:\Users\gualas\AppData\Local\Temp\ipykernel_3080\3674604142.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


In [44]:
poblacion_hombres=pd.read_csv(r'rangos/population-and-demography_male.csv')
poblacion_hombres.to_sql('hom',conn,index=False)
tabla_4.to_sql('t4',conn,index=False)

154

In [45]:
tabla_5=pd.read_sql('SELECT t4.*,hom.`Male population`,hom.`Male population aged 20 to 29 years`,hom.`Male population aged 30 to 39 years`,hom.`Male population aged 40 to 49 years`,hom.`Male population aged 50 to 59 years`,hom.`Male population aged 60 to 69 years` FROM t4 t4 JOIN hom hom ON(t4.Year=hom.Year AND t4.Country=hom.`Country name`)',conn)

verificar poblacion total

In [16]:
# Acceder a la página nro7 poblacion total
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)

# Abrir la página de la URL dada
driver.get("https://ourworldindata.org/world-population-growth")

# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/form/div[2]/div[2]/label/input')))
boton_descarga.click()

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()

time.sleep(2)
driver.quit()

C:\Users\gualas\AppData\Local\Temp\ipykernel_3080\1684948441.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',options=chrome_options)


In [46]:
pobrangent=pd.read_csv(r'rangos/population-and-demography_pobranget.csv')
pobrangent.to_sql('pob',conn,index=False)
tabla_5.to_sql('t5',conn,index=False)

154

In [47]:
tabla_6=pd.read_sql('SELECT t5.*,pob.`Population aged 20 to 29 years`,pob.`Population aged 30 to 39 years`,pob.`Population aged 40 to 49 years`,pob.`Population aged 50 to 59 years`,pob.`Population aged 60 to 69 years` FROM t5 t5 JOIN pob pob ON(t5.Year=pob.Year AND t5.Country=pob.`Country name`)',conn)

In [48]:
retornos=pd.read_csv(r'retornos\retornos_paises_3.csv')

In [49]:
retornos.to_sql('ret',conn,index=False)
tabla_6.to_sql('t6',conn,index=False)
tabla_7=pd.read_sql('SELECT t6.*,ret.`Returns by origin`,ret.`Returns by host` FROM t6 t6 JOIN ret ret ON(t6.Year=ret.Year AND t6.Country=ret.Country)',conn)

In [18]:
# Acceder a la página nro9 migrantes desaparecidos

url = 'https://missingmigrants.iom.int/sites/g/files/tmzbdl601/files/2023-02/Missing_Migrants_Global_Figures_allData_0.csv'

filename = os.path.join(os.path.expanduser("~"), "Desktop\prueba", "Missing_Migrants_Global_Figures_allData_0.csv")
urlretrieve(url, filename)



('C:\\Users\\gualas\\Desktop\\prueba\\Missing_Migrants_Global_Figures_allData_0.csv',
 <http.client.HTTPMessage at 0x1fa2e770fa0>)

In [50]:
desaparecidos=pd.read_csv(r'desaparecidos\Missing_Migrants_regiones.csv')

In [51]:
desaparecidos=desaparecidos[desaparecidos['Region of Incident'].isin(['North America','Central America','Caribbean' ,'South America'])]
desaparecidos=desaparecidos[desaparecidos['Region of Origin'].isin(['North America','Central America','Caribbean' ,'South America'])]

In [53]:
desaparecidos.to_sql('des',conn,index=False)

2048

In [54]:
des=pd.read_sql('SELECT `Region of Incident`,SUM(`Total Number of Dead and Missing`) as total,`Incident year`,`Region of Origin` FROM des GROUP BY `Incident year`, `Region of Incident`',conn)